In [1]:
%load_ext autoreload
%autoreload 2
import gym
import os
import numpy as np
import random
import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter
from experiment import rollout, ReplayBuffer, Trajectory, load_model, save_model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
MODEL_NAME = 'model_v0_lunar_lander_v2'
HIDDEN = 64
curr_step = 0
writer = SummaryWriter()

class Behavior(torch.nn.Module):
    def __init__(self, input_shape, num_actions):
        super(Behavior, self).__init__()
        self.classifier = torch.nn.Sequential(
            nn.Linear(input_shape, HIDDEN), 
            nn.Dropout(0.1),
            torch.nn.LayerNorm(HIDDEN),
            nn.ReLU(),
            nn.Linear(HIDDEN, HIDDEN),
            torch.nn.LayerNorm(HIDDEN),
            nn.Dropout(0.1),
            nn.ReLU(),            
            nn.Linear(HIDDEN, HIDDEN), 
            torch.nn.LayerNorm(HIDDEN),
            nn.Dropout(0.1),
            nn.ReLU(),            
            nn.Linear(HIDDEN, HIDDEN), 
            torch.nn.LayerNorm(HIDDEN),
            nn.Dropout(0.1),
            nn.ReLU(),
            nn.Linear(HIDDEN, num_actions)
        )        

    def forward(self, x):
        return self.classifier(x)

In [3]:
env = gym.make('LunarLander-v2')

In [4]:
loss_object = torch.nn.CrossEntropyLoss().to(device)
model_sample = Behavior(input_shape=env.observation_space.shape[0]+2, num_actions=env.action_space.n).to(device)
optimizer = torch.optim.Adam(model_sample.parameters(), lr=0.01)

In [5]:
rb = ReplayBuffer(max_size=50, last_few=25)

n_warmup_episodes = 10
# Random rollout
trajectories, mean_reward, length = rollout(episodes=n_warmup_episodes, env=env, render=False)
rb.add(trajectories)

# Keep track of steps used during random rollout!
epoch, model_sample, optimizer, loss, steps = load_model(MODEL_NAME, model_sample, optimizer, device, train=True)
steps += length
save_model(MODEL_NAME, epoch, model_sample, optimizer, loss, steps)

# Plot initial values
writer.add_scalar('Steps/reward', mean_reward, steps)        

No checkpoint found. Creating new model.


In [6]:
batch_size = 1024

In [7]:
def train_step(model, inputs, targets):
    optimizer.zero_grad()    
    predictions = model(inputs)
    loss = loss_object(predictions, targets)
    
    loss.backward()
    optimizer.step()
    
    return loss

def action_fn(model, inputs, sample_action=True):
    action_logits = model(inputs)
    action_probs = torch.softmax(action_logits, axis=-1)

    if sample_action:        
        m = torch.distributions.categorical.Categorical(logits=action_logits)             
        action = int(m.sample().squeeze().cpu().numpy())        
    else:
        action = int(np.argmax(action_probs.detach().squeeze().numpy()))
    return action
    

In [8]:
# SAMPLE ACTIONS

loss_sum = 0
loss_count = 0

epochs = 1000000
epoch, model_sample, optimizer, loss, steps = load_model(MODEL_NAME, model_sample, optimizer, device, train=True)
print(steps)

eval_every = 2000

for i in range(epoch, epochs+epoch):
    x, y = rb.sample(batch_size, device)    
    loss = train_step(model_sample, x, y)
    loss_sum += loss
    loss_count += 1
    
    writer.add_scalar('Loss/loss', loss, i)
    
    (dh, dr) = rb.sample_command()
    writer.add_scalar('Epoch/dh', dh, i)
    writer.add_scalar('Epoch/dr', dr, i)

    n_episodes_per_iter = 10
    n_updates_per_iter = 1_000
    if i % n_updates_per_iter == 0:
        trajectories, mean_reward, length = rollout(n_episodes_per_iter, env=env, model=model_sample, sample_action=True, replay_buffer=rb, 
                              device=device, action_fn=action_fn)
        rb.add(trajectories)
        
        steps += length
        avg_loss = loss_sum/loss_count
        save_model(MODEL_NAME, i, model_sample, optimizer, avg_loss, steps)        
        print(f"Average Episode Reward: {mean_reward}")        
        writer.add_scalar('Steps/reward', mean_reward, steps)
        
        mean_length = length*1.0/n_episodes_per_iter
        writer.add_scalar('Steps/length', mean_length, steps)
        
        
    if i % eval_every == 0:
        eval_episodes = 10
        _, mean_reward, length = rollout(eval_episodes, env=env, model=model_sample, 
                            sample_action=True, replay_buffer=rb, 
                            device=device, action_fn=action_fn)
        
        writer.add_scalar('Epoch/reward', mean_reward, i)        
        mean_length = length*1.0/n_episodes_per_iter
        writer.add_scalar('Epoch/length', mean_length, i)
        
    if i % 200 == 0:
        avg_loss = loss_sum/loss_count
        print(f'i: {i}, s: {steps}, Loss: {avg_loss}')
        
        save_model(MODEL_NAME, i, model_sample, optimizer, avg_loss, steps)

Existing model found. Loading from epoch 0, steps 906 with loss: 0.0
906
Average Episode Reward: -139.0475068970673
i: 0, s: 1605, Loss: 1.4952491521835327
i: 200, s: 1605, Loss: 1.3297474384307861
i: 400, s: 1605, Loss: 1.31969153881073
i: 600, s: 1605, Loss: 1.3113675117492676
i: 800, s: 1605, Loss: 1.3041019439697266
Average Episode Reward: -133.93800133332016
i: 1000, s: 2494, Loss: 1.2972888946533203
i: 1200, s: 2494, Loss: 1.295034646987915
i: 1400, s: 2494, Loss: 1.2920233011245728
i: 1600, s: 2494, Loss: 1.2891091108322144
i: 1800, s: 2494, Loss: 1.2859865427017212
Average Episode Reward: -204.9974107568035
i: 2000, s: 3377, Loss: 1.283085584640503
i: 2200, s: 3377, Loss: 1.2823238372802734
i: 2400, s: 3377, Loss: 1.2813516855239868
i: 2600, s: 3377, Loss: 1.280329704284668
i: 2800, s: 3377, Loss: 1.2791187763214111
Average Episode Reward: -161.13584940839596
i: 3000, s: 4265, Loss: 1.2779054641723633
i: 3200, s: 4265, Loss: 1.2776564359664917
i: 3400, s: 4265, Loss: 1.27724874

KeyboardInterrupt: 

In [9]:
rb.sample_command()

(418.8, 212.88026653543906)

In [13]:
cmd = (280.085, 318.91295076177306)
rb.sample_command()
env = gym.make('LunarLander-v2')
e, model, _, l,_ = load_model(name=MODEL_NAME, train=False, model=model_sample, optimizer=optimizer, device=device)

_, mean_reward, _ = rollout(episodes=5, env=env, model=model_sample, sample_action=False, 
                      cmd=cmd, render=True, device=device, action_fn=action_fn)


print(f"Average Episode Reward: {mean_reward}")

Existing model found. Loading from epoch 643400, steps 669481 with loss: 0.9160314798355103
Average Episode Reward: -152.900474090895
